# Importación de librerias
---



In [1]:
# Importe de librerias
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime
import datetime as dtime
import matplotlib.dates as mdates

from keras import Sequential
from keras.layers import Dense

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from keras import Sequential
from keras.layers import Dense

Using TensorFlow backend.


# Dataset

In [0]:
# Se carga la información en un dataframe df
df = pd.read_csv('train.csv', low_memory=False)

id                   91433 non-null object > N
LoanNr_ChkDgt        91433 non-null int64  > N
Name                 91433 non-null object > N
City                 91427 non-null object > N
State                91433 non-null object > N
Zip                  91433 non-null int64  > S
Bank                 91335 non-null object > N
BankState            91334 non-null object > N
NAICS                91433 non-null int64  > S
ApprovalDate         91433 non-null object > N
ApprovalFY           91433 non-null object > N
Term                 91433 non-null int64  > S
NoEmp                91433 non-null int64 
NewExist             91427 non-null float64> S
CreateJob            91433 non-null int64  > N
RetainedJob          91433 non-null int64  > N
FranchiseCode        91433 non-null int64  > N
UrbanRural           91433 non-null int64  > S/N
RevLineCr            91088 non-null object > S/N
LowDoc               91317 non-null object > S
ChgOffDate           17449 non-null object > S
DisbursementDate     91220 non-null object > S
DisbursementGross    91433 non-null object > S
BalanceGross         91433 non-null object > ?
Accept               91433 non-null int64  <=T


# Edicion de datos


RevLineCr: tiene varios valores posibles  '

```
# This is formatted as code
```
0    31997

1        4

4        1

N    36553

T     1642

Y    20888

`        3 

No es posible inferir del dataset si un 0 significa un N o un Y. Por tanto no se va a usar este dato.

df.groupby('NewExist').size() '

df['NewExist'].describe()

In [3]:
# LIMPIEZA DE DATOS

# Se asigna el valor de 0 a los registros vaciós en la columna 'NewExist'
df['NewExist']=df['NewExist'].apply(lambda x: 0 if pd.isnull(x) else x)
df['NewExist']=df['NewExist'].astype(int)
df['UrbanRural']=df['UrbanRural'].astype(int)
# Se quita el simbolo de dollar y signos de puntuación
df['DisbursementGross']=df['DisbursementGross'].str.replace('$','')
df['DisbursementGross']=df['DisbursementGross'].str.replace(',','')
df['DisbursementGross']=df['DisbursementGross'].str.replace('\.00','')
df['DisbursementGross']=df['DisbursementGross'].astype(int)

# Se quita el simbolo de dollar y signos de puntuación
df['BalanceGross']=df['BalanceGross'].str.replace('$','')
df['BalanceGross']=df['BalanceGross'].str.replace(',','')
df['BalanceGross']=df['BalanceGross'].str.replace('\.00','')
df['BalanceGross']=df['BalanceGross'].astype(int)

# Se cambia a formtao fecha la columna DisbursementDate para posteriormente realizar operaciones
df['DisbursementDate']=df[df['DisbursementDate'].notnull()].DisbursementDate.apply(lambda x: dtime.datetime.strptime(x,'%d-%b-%y'))
df['ApprovalDate']=df[df['ApprovalDate'].notnull()].ApprovalDate.apply(lambda x: dtime.datetime.strptime(x,'%d-%b-%y'))
df['ChgOffDate']=df[df['ChgOffDate'].notnull()].ChgOffDate.apply(lambda x: dtime.datetime.strptime(x,'%d-%b-%y'))


# Nos quedamos solo con los valores numéricos
df['ApprovalFY']=df['ApprovalFY'].str.extract('(\d+)')
df['ApprovalFY']=df['ApprovalFY'].str.extract('(\d+)').astype(int)
#df['ApprovalFY']=df['ApprovalFY'].apply(lambda x: dtime.datetime.strptime(x,'%Y'))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


In [0]:
# NAICS contiene información importante referente a la industria a la que pertenece la empresa. Creamos una nueva columna con los dos dígitos de la izquierda
df['Sector']=(df.NAICS/1000).astype(int)
df['Respaldado']=df['Term'].map(lambda x: 1 if x >=240 else 0)

Agregamos nueva información
*   Crisis (Si la fecha de aprovación del credito esta entre jul-2007 y jun-2009)
*   List item



In [0]:
# Averiguar si el credito fue aprovado durante la recesion
#df['Recesion']=df['DisbursementDate'].map(lambda x: 1 if (x>dtime.datetime(2007, 12, 1, 0, 0) and x < dtime.datetime(2009, 6, 30, 0, 0)) else 0)
#df['Crisis']=df['DisbursementDate'].map(lambda x: 1 if (x>dtime.datetime(2007, 7, 1, 0, 0) and x < dtime.datetime(2009, 6, 30, 0, 0)) else 0)
df['Crisis']=df['ApprovalDate'].map(lambda x: 1 if (x>dtime.datetime(2007, 12, 1, 0, 0) and x < dtime.datetime(2009, 6, 30, 0, 0)) else 0)

In [0]:
# El plazo del crédito en número de días
df['TermEnDias']=df['Term'].apply(lambda x: pd.Timedelta(days=x*30))

In [0]:
# Creditos que estuvieron activos durante la recesión
df['ActivoRecesion']=((df['DisbursementDate']+df['TermEnDias'])>dtime.datetime(2007, 7, 1, 0, 0))
df['DespuesRecesion']=((df['DisbursementDate']+df['TermEnDias'])<dtime.datetime(2009, 6, 30, 0, 0))

In [0]:
#
df['EntreRecesion']=(df['DespuesRecesion'] == df['ActivoRecesion'])
df['DespuesRecesion']=df['DespuesRecesion'].astype(int)
df['ActivoRecesion']=df['ActivoRecesion'].astype(int)
df['EntreRecesion']=df['EntreRecesion'].astype(int)

In [0]:
# Muestra si tiene fecha de cambio en ChgOffDate
df['Default']=df.ChgOffDate.notnull().astype(int)

# Exploracion de datos

In [0]:
#df.corr()
#sns.heatmap(df.corr(), annot=False)

# Deducciones:
# - Term y DisbursementGross parecen ser las más relevantes para la clasificación. Además, amabs están fuertemente correladas entre sí.
# - Parece que el hecho de que sea Urban o Rural o NewExit no tienen ninguna importancia con darle el crédito, si lo quitásemos probablemente no perdería accuracy.
# - ...

In [11]:
df.corr().style.background_gradient(cmap='seismic')

/usr/local/lib/python3.6/dist-packages/matplotlib/colors.py:512: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


,LoanNr_ChkDgt,Zip,NAICS,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,DisbursementGross,BalanceGross,Accept,Sector,Respaldado,Crisis,ActivoRecesion,DespuesRecesion,EntreRecesion,Default
LoanNr_ChkDgt,1,0.00920972,-0.0278598,-0.14954,0.127793,0.0171996,-0.0429607,-0.0217534,-0.021912,0.0416751,-0.114621,0.0931493,nan,0.203724,-0.0278352,0.10527,-0.130622,-0.114115,0.0387681,-0.104147,-0.204226
Zip,0.00920972,1,-0.0119565,-0.0427463,0.0310284,-0.00133835,0.00337901,0.00204138,0.000852684,0.0246083,-0.0108146,0.00434109,nan,0.0428364,-0.0119266,0.0238247,-0.00418902,0.00152149,-0.00537355,-0.00663525,-0.0465891
NAICS,-0.0278598,-0.0119565,1,0.483835,-0.193447,-0.0440909,0.0645264,-0.0403944,-0.037814,0.0276015,0.44788,-0.0652543,nan,-0.151563,0.999999,-0.168807,0.09544,0.231552,-0.191416,0.0275967,0.152896
ApprovalFY,-0.14954,-0.0427463,0.483835,1,-0.396368,-0.049797,0.060559,-0.0672741,-0.0604689,-0.00934916,0.773731,-0.0756118,nan,-0.310979,0.48367,-0.316219,0.256237,0.394115,-0.337853,0.0340695,0.312872
Term,0.127793,0.0310284,-0.193447,-0.396368,1,0.0547139,-0.123049,0.0225582,0.020708,-0.0126947,-0.30761,0.531799,nan,0.41751,-0.193341,0.931765,-0.102608,0.370577,-0.460017,-0.211639,-0.425792
NoEmp,0.0171996,-0.00133835,-0.0440909,-0.049797,0.0547139,1,-0.0369883,0.0120105,0.0223673,-0.00296862,-0.0447796,0.094089,nan,0.0390302,-0.0440945,0.0621983,-0.00847784,-0.0012645,-0.00340982,-0.00843759,-0.0401653
NewExist,-0.0429607,0.00337901,0.0645264,0.060559,-0.123049,-0.0369883,1,-0.00754431,-0.0123446,0.138827,0.04037,-0.0967308,nan,-0.0540357,0.0645758,-0.15726,0.047502,-0.0267823,0.0315333,0.012219,0.0589002
CreateJob,-0.0217534,0.00204138,-0.0403944,-0.0672741,0.0225582,0.0120105,-0.00754431,1,0.987489,0.000927263,-0.0300458,0.00695474,nan,0.00327481,-0.0403817,0.0345849,-0.00436862,-0.0394908,0.031077,-0.00477106,-0.00281777
RetainedJob,-0.021912,0.000852684,-0.037814,-0.0604689,0.020708,0.0223673,-0.0123446,0.987489,1,7.19349e-05,-0.0224593,0.0118966,nan,0.00342629,-0.0378038,0.0327317,-0.00353888,-0.0359267,0.0282513,-0.00413014,-0.00319146
FranchiseCode,0.0416751,0.0246083,0.0276015,-0.00934916,-0.0126947,-0.00296862,0.138827,0.000927263,7.19349e-05,1,0.00360874,0.0303512,nan,0.0177118,0.0276889,-0.0475765,-0.00680903,-0.00702935,0.00641963,-0.00121145,-0.018268


In [0]:
#sns.pairplot(df, hue='Accept', diag_kind = 'kde', plot_kws = {'alpha': 0.6, 's': 80, 'edgecolor': 'k'}, size = 4)

# Deducciones:
# - Sólo 1/5 parte no han sido aceptadas (0) eso propicia overfitting hacia el sí (1).
# - Vemos que cuanto más alto es el préstamos pedido, y cuanto más bajo es el término para devolverlo, más difícil es que lo acepten. 
# - ...

# Aprendizaje automatico

In [0]:
#columns = ['Zip', 'Term', 'NoEmp', 'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural',  'DisbursementGross']
columns = ['Term', 'UrbanRural', 'DisbursementGross','NAICS','Default','Respaldado','Crisis']
#columns = ['Default','Crisis']
X= df[columns] 
y= df['Accept']


In [14]:
#standardizing the input feature
sc = StandardScaler()
mm =  MinMaxScaler()
rs = RobustScaler()
X = sc.fit_transform(X)
#X = mm.fit_transform(X)
#X = rs.fit_transform(X)
X

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


array([[ 1.63016752,  0.58422145, -0.53093818, ..., -0.48564214,
         1.48534943, -0.2016899 ],
       [-0.22668471, -1.23102032, -0.20442815, ..., -0.48564214,
        -0.67324225, -0.2016899 ],
       [-1.25826928,  0.58422145, -0.6802169 , ...,  2.05912937,
        -0.67324225, -0.2016899 ],
       ...,
       [-1.17574252,  0.58422145, -0.76942728, ...,  2.05912937,
        -0.67324225, -0.2016899 ],
       [-0.84563545,  0.58422145, -0.6504801 , ...,  2.05912937,
        -0.67324225, -0.2016899 ],
       [-0.65995023,  0.58422145,  0.03346622, ...,  2.05912937,
        -0.67324225, -0.2016899 ]])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=33)
print(f"{X_train.size} {X_test.size} {y_test.size} {y_test.size}")

544026 96005 13715 13715


In [16]:
##
##  MULTI-LAYER PERCEPTRON (MLP) - 100% accuracy   /:-|
##

# Vamos a construir la red neuronal. Utilizo la la biblioteca keras de Google que es la más utilizada.

# Esto es algo así como el modelo, la red completa.
classifier = Sequential()

# Capa oculta 1
classifier.add(Dense(8, activation='relu',  input_dim=7))

# Capa oculta 2
classifier.add(Dense(8, activation='relu', kernel_initializer='random_normal'))

# Capa oculta 3
classifier.add(Dense(8, activation='relu', kernel_initializer='random_normal'))


# Capa de salida
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

# Compilamos la red neuronal
classifier.compile(optimizer ='rmsprop',loss='binary_crossentropy', metrics =['mae','accuracy'])

# Entrenamos el modelo con el dataset de entrenamiento escogido. 
classifier.fit(X_train,y_train, batch_size=10, epochs=20)

#cada epoch es una iteración de todo el dataset de entranmiento en el proceso
# de backpropagation, y que la distancia hasta el error (mediante el descenso del gradiente) va 
# disminuyendo de forma inversa a la precisión que va aumentando logicamente. 


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
77718/77718 [==============================] - 8s 103us/step - loss: 0.0600 - mean_absolute_error: 0.0227 - acc: 0.9875
Epoch 2/20
77718/77718 [==============================] - 8s 98us/step - loss: 0.0445 - mean_absolute_error: 0.0086 - acc: 0.9933
Epoch 3/20
77718/77718 [==============================] - 8s 100us/step - loss: 0.0444 - mean_absolute_error: 0.0087 - acc: 0.9933
Epoch 4/20
77718/77718 [==============================] - 8s 103us/step - loss: 0.0444 - mean_absolute_error: 0.0086 - acc: 0.9933
Epoch 5/20
77718/77718 [==============================] - 8s 99us/step - loss: 0.0443 - mean_absolute_error: 0.0087 - acc: 0.9933
Epoch 6/20
77718/77718 [==============================] - 8s 98us/step - loss: 0.0440 - mean_absolute_error: 0.0089 - acc: 0.9933
Epoch 7/20
77718/77718 [==============================] - 8s 98us/step - loss: 0.0445 - mean_abso

In [17]:
eval_model=classifier.evaluate(X_train, y_train)
eval_model

77718/77718 [==============================] - 1s 17us/step


[0.041407624771136686, 0.01034268373273816, 0.9933348773771842]

# TEST

In [0]:
df_test = pd.read_csv('test_nolabel.csv', low_memory=False)

In [0]:
df_test['NewExist']=df_test['NewExist'].apply(lambda x: 0 if pd.isnull(x) else x)
#df_test.groupby('NewExist').size()
#df_test['NewExist'].describe()
#df_test['NewExist'].notnull().sum()
#df_test['NewExist']
#df_test[df_test['NewExist'].isnull()].NewExist
#df_test[671:672:2].NewExist


In [0]:
# TEST
df_test['Sector']=(df_test.NAICS/1000).astype(int)
df_test['Respaldado']=df_test['Term'].map(lambda x: 1 if x >=240 else 0)
df_test['Default']=df_test.ChgOffDate.notnull().astype(int)
# Se quita el simbolo de dollar y signos de puntuación
df_test['DisbursementGross']=df_test['DisbursementGross'].str.replace('$','')
df_test['DisbursementGross']=df_test['DisbursementGross'].str.replace(',','')
df_test['DisbursementGross']=df_test['DisbursementGross'].str.replace('\.00','')
df_test['DisbursementGross']=df_test['DisbursementGross'].astype(int)


In [0]:
df_test['ApprovalDate']=df_test[df_test['ApprovalDate'].notnull()].ApprovalDate.apply(lambda x: dtime.datetime.strptime(x,'%d-%b-%y'))
df_test['Crisis']=df_test['ApprovalDate'].map(lambda x: 1 if (x>dtime.datetime(2007, 12, 1, 0, 0) and x < dtime.datetime(2009, 6, 30, 0, 0)) else 0)

In [22]:
#columns = ['Term', 'UrbanRural',  'DisbursementGross','Respaldado','Default']
df_test2= df_test[columns]
df_test2 = sc.fit_transform(df_test2)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [0]:
y_pred=classifier.predict(df_test2)
y2 = pd.DataFrame({'Col':y_pred[:,0]})

In [24]:
y2.loc[y2["Col"] >= 0.5, "Col"] = "1"
y2.loc[y2["Col"] != '1', "Col"] = "0"
print(f"{df_test['id'].shape} {y2['Col'].shape}")

(14574,) (14574,)


In [25]:
ypd = pd.DataFrame({'id':df_test['id'], 'Accept':y2['Col']})
ypd.to_csv('Predictions2-21032019.csv',encoding='utf-8', index=False)
print(f"{df_test['id'].shape} {y2['Col'].shape}")

(14574,) (14574,)
